## Import Wheat Solver Excel Template Data into a pandas dataframe

In [1]:
import pandas as pd
import numpy as np

In [2]:
# read in excel solver spreadsheet as pandas dataframe.

df = pd.read_excel(r'C:\Users\rj71b\contract_allocation\data\interim\Wheat_Solver_Advanced.xlsx',
                  sheet_name = 0,
                  skiprows = 2,
                  nrows = 4,
                  usecols = [x for x in range(2,19)])

In [3]:
# drop columns with NAN. Return df with only contract details
df.dropna(axis = 1,
                   inplace = True)

In [4]:
# Reflect the DataFrame over its main diagonal by writing 
# rows as columns and vice-versa

df = df.T

In [5]:
# Reset the index of the DataFrame, and use the default one instead

df.reset_index(drop = True, inplace = True)

In [6]:
# set column names to those in the first row
    
df.columns = df.iloc[0]

In [7]:
df = df.reindex(df.index.drop(0))

In [8]:
# initial dataframe with contract details
df

,Merchant,Contract #,Base,Tonnes
1,CBH,WC1,APW1,250
2,Bunge,WC2,APW1,250
3,Glencore,WC3,APW1,500
4,CBH,WC4,APW1,500
5,AWB,WC5,APW1,500
6,CBH,WC6,APW1,500


### For now am just creating tonnes to allocate to cash df. 
### Will work out how to incorporate from xcel import later

In [9]:
# data dict
data = {'Merchant':[np.NAN], 'Base':['APW1'], 'Tonnes':5000}
# convert to pandas dataframe
df_cash_contract =  pd.DataFrame(data, index =['cash']) 

In [10]:
# Make a deep copy of df and assign as df_contract. Am doing this so as to set index on Contract #. Will allow for indexing later on
df_contract = df.copy()

In [11]:
# Have set contract number column as index
df_contract.set_index('Contract #', inplace = True)

In [12]:
df_contract.index.name = 'contract'

In [13]:
df_contract = df_contract.append(df_cash_contract)

In [14]:
# now have contract and cash tonnes for allocation
df_contract

,Merchant,Base,Tonnes
WC1,CBH,APW1,250
WC2,Bunge,APW1,250
WC3,Glencore,APW1,500
WC4,CBH,APW1,500
WC5,AWB,APW1,500
WC6,CBH,APW1,500
cash,NaN,APW1,5000


## Will now get dataframe with wheat grades and ammount grown

In [15]:
df_grown = pd.read_excel(r'C:\Users\rj71b\contract_allocation\data\interim\Wheat_Solver_Advanced.xlsx',
                  sheet_name = 0,
                  skiprows = 22,
                  usecols = [1,2],
                  nrows = 9,
                  header = None,
                  names = ['grade', 'grown'],
                  index_col = 0)

In [16]:
df_grown

,grown
grade,
H1,750
H2,150
APW1,2500
APW2,550
AUH2,250
ASW1,2000
AGP1,500
FED1,800
APWN,0


## Create df with contract values

In [17]:
df_values = pd.read_excel(r'C:\Users\rj71b\contract_allocation\data\interim\Wheat_Solver_Advanced.xlsx',
                  sheet_name = 0,
                  skiprows = 22,
                  usecols = [x for x in range(3,3+len(df['Contract #']))],
                  nrows = 9,
                  header = None,
                  names = df['Contract #'].to_list())  # column names are contract numbers

In [18]:
df_values = df_values.set_index(df_grown.index)

In [19]:
df_values

,WC1,WC2,WC3,WC4,WC5,WC6
grade,,,,,,
H1,300,307,310,313,311,317
H2,298,303,305,308,308,312
APW1,295,297,299,302,305,308
APW2,280,292,290,297,302,306
AUH2,290,280,301,305,300,309
ASW1,265,282,279,284,295,285
AGP1,260,257,260,265,270,269
FED1,235,227,230,252,235,236
APWN,295,298,302,302,307,304


## Create df with cash values

In [20]:
df_cash = pd.read_excel(r'C:\Users\rj71b\contract_allocation\data\interim\Wheat_Solver_Advanced.xlsx',
                  sheet_name = 0,
                  skiprows = 22,
                  usecols = [1,18],
                  nrows = 9,
                  header = None,
                  names = ['grade', 'cash'],
                  index_col = 0)

In [21]:
df_cash

,cash
grade,
H1,385
H2,380
APW1,343
APW2,322
AUH2,350
ASW1,315
AGP1,313
FED1,260
APWN,343


## merge df_values and df_cash to create one df for use in optimistion

In [22]:
df_merge = pd.merge(df_values, df_cash, left_index=True, right_index=True)

In [23]:
df_merge

,WC1,WC2,WC3,WC4,WC5,WC6,cash
grade,,,,,,,
H1,300,307,310,313,311,317,385
H2,298,303,305,308,308,312,380
APW1,295,297,299,302,305,308,343
APW2,280,292,290,297,302,306,322
AUH2,290,280,301,305,300,309,350
ASW1,265,282,279,284,295,285,315
AGP1,260,257,260,265,270,269,313
FED1,235,227,230,252,235,236,260
APWN,295,298,302,302,307,304,343


## Wheat solver excel spreadsheet now imported in as pandas dataframes.
### Ready to start Linear Programming for Multi grade wheat contracts using Pulp and pandas

In [24]:
from pulp import *

In [45]:
# Instantiate our problem class
model = pulp.LpProblem("Profit maximise wheat allocation", pulp.LpMaximize)

In [46]:
# construct our dicision varable lists

# [j for j in df_merge.index]  give a list of grades

# [i for i in df_merge.columns] gives a list of contract and cash names

In [47]:
# Each of these decision variables will have similar characteristics (lower bound of 0, continuous variables). 
# Therefore we can use PuLP’s LpVariable object’s dict functionality, we can provide our tuple indices.
# These tuples will be keys for the weight_tonnes dict of decision variables

grade_weight = pulp.LpVariable.dicts("weight tonnes",
                                     ((i, j) for i in df_merge.columns for j in df_merge.index),
                                     lowBound=0,
                                     cat='Integer')

In [48]:
# Objective Function is added to 'model' first


model += (
    pulp.lpSum([grade_weight[contract, grade]*df_merge[contract][grade] for contract in df_merge.columns for grade in df_merge.index])), "Total profit from selling all wheat"


In [49]:
# add constraints for contracts
for contract in df_contract.index.values:
    model += pulp.lpSum([grade_weight[contract, grade] for grade in df_merge.index]) == df_contract.loc[contract]['Tonnes'], f"{contract} requirement"
    

In [50]:
# constraints for grades grown
for grade in df_grown.index:
    model += pulp.lpSum([grade_weight[contract, grade] for contract in df_contract.index.values]) <= df_grown.loc[grade]

In [51]:
print(model)

Profit maximise wheat allocation:
MAXIMIZE
260*weight_tonnes_('WC1',_'AGP1') + 295*weight_tonnes_('WC1',_'APW1') + 280*weight_tonnes_('WC1',_'APW2') + 295*weight_tonnes_('WC1',_'APWN') + 265*weight_tonnes_('WC1',_'ASW1') + 290*weight_tonnes_('WC1',_'AUH2') + 235*weight_tonnes_('WC1',_'FED1') + 300*weight_tonnes_('WC1',_'H1') + 298*weight_tonnes_('WC1',_'H2') + 257*weight_tonnes_('WC2',_'AGP1') + 297*weight_tonnes_('WC2',_'APW1') + 292*weight_tonnes_('WC2',_'APW2') + 298*weight_tonnes_('WC2',_'APWN') + 282*weight_tonnes_('WC2',_'ASW1') + 280*weight_tonnes_('WC2',_'AUH2') + 227*weight_tonnes_('WC2',_'FED1') + 307*weight_tonnes_('WC2',_'H1') + 303*weight_tonnes_('WC2',_'H2') + 260*weight_tonnes_('WC3',_'AGP1') + 299*weight_tonnes_('WC3',_'APW1') + 290*weight_tonnes_('WC3',_'APW2') + 302*weight_tonnes_('WC3',_'APWN') + 279*weight_tonnes_('WC3',_'ASW1') + 301*weight_tonnes_('WC3',_'AUH2') + 230*weight_tonnes_('WC3',_'FED1') + 310*weight_tonnes_('WC3',_'H1') + 305*weight_tonnes_('WC3',_'H2')

In [52]:
# Solve our problem
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [53]:
for var in grade_weight:
    if grade_weight[var].varValue > 0:
        var_value = grade_weight[var].varValue
        print("The weight of {0} in {1} wheat is {2} tonnes".format(var[1], var[0], var_value))

The weight of FED1 in WC1 wheat is 250.0 tonnes
The weight of ASW1 in WC2 wheat is 250.0 tonnes
The weight of APW2 in WC3 wheat is 50.0 tonnes
The weight of ASW1 in WC3 wheat is 400.0 tonnes
The weight of FED1 in WC3 wheat is 50.0 tonnes
The weight of FED1 in WC4 wheat is 500.0 tonnes
The weight of ASW1 in WC5 wheat is 500.0 tonnes
The weight of APW2 in WC6 wheat is 500.0 tonnes
The weight of H1 in cash wheat is 750.0 tonnes
The weight of H2 in cash wheat is 150.0 tonnes
The weight of APW1 in cash wheat is 2500.0 tonnes
The weight of AUH2 in cash wheat is 250.0 tonnes
The weight of ASW1 in cash wheat is 850.0 tonnes
The weight of AGP1 in cash wheat is 500.0 tonnes


In [54]:
total_profit = pulp.value(model.objective)

print("The total profit is ${} for contracts".format(round(total_profit, 0)))

The total profit is $2408350.0 for contracts
